## STRUCTS

sizeof() se puede utilizar para obtener el tamaño de bytes que ocupa en memoria una clase, struct o variable.

In [1]:
struct Ejemplo {
    int num;
    char c;
};

printf("MAIN\n=======\n");
Ejemplo ej;
printf("%lu", sizeof(Ejemplo));

return 0;


MAIN
8

Estamos en un caso en el que el compilador ha añadido padding, espacio vacío extra para que los datos de nuestro struct estén alineados a la palabra del procesador. El procesador es mucho más rápido accediendo a memoria alineada. 

Cada tipo tiene un padding asociado a él, esto se traduce a “el espacio de memoria mínimo que tiene que haber entre dos elementos del mismo tipo”.

El orden de elementos hace que structs con los mismos datos, ocupen más. Es muy importante siempre al crear structs grandes, ordenar los elementos de mayor a menor(o viceversa) para evitar espacio extra por el padding.

Podemos sobreescribir el padding:

In [2]:
//pragma pack(1) //VISUAL STUDIO
struct Ejemplo {
    int num;
    char c;
};
// __attribute__((packed)) //CLANG y GCC

printf("MAIN\n=======\n");
Ejemplo ej;
printf("%lu", sizeof(Ejemplo));

return 0;


MAIN
8

## UNIONS

Los elementos de una Union no estan seguidos, sino que ocupan la misma memoria.

Utilidad:
- Polimorfismo de objetos sin crear indirecciones o funciones virtuales.
- Reutilización de memoria sin hacer reinterpr_cast o similares.

Solo puede estar *un solo* elemento activo de un union al mismo tiempo, en C++, acceder a un elemento de la union que no este activo se considera undefined behavior!!

Las unions y los structs pueden combinarse perfectamente. Podemos declarar una struct sin nombre dentro del union
para indicar que esos elementos van a estar en secuencia y al revés.

## CLASES

Una clase, a efectos teóricos, es un struct cuyos elementos pueden
tener private, protected, public para poder acceder a ellos y puede contener métodos.

Sin embargo, en C++ un struct una clase es exactamente lo mismo, la única diferencia es la visibilidad por defecto. En class es private y en struct public.

Las clases (y structs) de C++ tienen la particularidad de que sus
métodos pueden ser const.

Un método const es un método que no puede modificar ninguno de los
datos de la instancia:

In [4]:
class MyClass 
{
    public:
    int GetScore() const
    {
        a += 1;
       return a;    
    }
    
    private:
        int a;
};

MyClass obj;
int score = obj.GetScore();

input_line_10:6:11: error: cannot assign to non-static data member within const member function 'GetScore'
        a += 1;
        ~ ^
input_line_10:4:9: note: member function 'MyClass::GetScore' is declared const here
    int GetScore() const
    ~~~~^~~~~~~~~~~~~~~~


Interpreter Error: 

Si tenemos una instancia const de una clase. Solamente podremos usar métodos que son const ya que el objeto no podemos modificarlo.

In [5]:
class MyClass 
{
    public:
    MyClass() 
    {
        score = 0;
    }
    void AddScore() 
    {
        score += 1;
    }
    
    int GetScore() const
    {
       return score;    
    }
    
    private:
        int score;
};

const MyClass obj;
int score = obj.GetScore();
obj.AddScore();

input_line_11:24:1: error: 'this' argument to member function 'AddScore' has type 'const __cling_N56::MyClass', but function is not marked const
obj.AddScore();
^~~
input_line_11:8:10: note: 'AddScore' declared here
    void AddScore() 
         ^


Interpreter Error: 

Disponemos de una inicialización que ocurre antes del cuerpo del constructor que se llama member initializer. (Solo de esta manera podemos inicializar variables constantes)

In [6]:
class MyClass 
{
    public:
    MyClass()
    {
        score = 0;
        other = 0;
    }
    
    private:
        int score;
        const int other;
};

input_line_12:4:5: error: constructor for 'MyClass' must explicitly initialize the const member 'other'
    MyClass()
    ^
input_line_12:12:19: note: declared here
        const int other;
                  ^
input_line_12:7:15: error: cannot assign to non-static data member 'other' with const-qualified type 'const int'
        other = 0;
        ~~~~~ ^
input_line_12:12:19: note: non-static data member 'other' declared const here
        const int other;
        ~~~~~~~~~~^~~~~


Interpreter Error: 

In [7]:
class MyClass 
{
    public:
    MyClass() : score(0), other(0)
    {
    }
    
    private:
        int score;
        const int other;
};

También podemos asignar el valor inicial por defecto en la propia
variable. Lo cual es aún más legible:

In [8]:
class MyClass 
{
    public:
    MyClass()
    {
        score = 0;
        other = 0;
    }
    
    private:
    int score;
    const int other;
};

input_line_14:4:5: error: constructor for 'MyClass' must explicitly initialize the const member 'other'
    MyClass()
    ^
input_line_14:12:15: note: declared here
    const int other;
              ^
input_line_14:7:15: error: cannot assign to non-static data member 'other' with const-qualified type 'const int'
        other = 0;
        ~~~~~ ^
input_line_14:12:15: note: non-static data member 'other' declared const here
    const int other;
    ~~~~~~~~~~^~~~~


Interpreter Error: 

## HERENCIA

Si no ponemos nada, se asumirá la visibilidad por defecto. La cual es private si el padre es una clase, y public si es un struct. Los member initializer también deben de llamar a los constructores de clases padre, proveyendo los argumentos de construcción:

In [9]:
class VirtualClass 
{
    public:
    VirtualClass(int initialScore) : score(initialScore)
    {
    }
    
    virtual void func() 
    {
        printf("padre");
    }
    
    private:
    int score;
};

class VirtualSubClass : public VirtualClass 
{
    public:
    VirtualSubClass() : VirtualClass(0) //score = 0
    {
    } 
    
    virtual void func() override 
    {
        printf("hijo\n");
    }
};

VirtualSubClass subclass;
subclass.func();

hijo


#### POLIMORFISMO

In [10]:
//Puntero
VirtualSubClass subclass;
VirtualClass* ptr = &subclass;
ptr->func();

//Alias
VirtualClass& pRef = subclass;
pRef.func();

hijo
hijo


En el momento que abstraemos Hijo como Padre, C++ no sabe realmente qué objeto estamos tratando de acceder mediante func(), imprimirá “Padre”, o “Hijo” ?
Para solucionar este problema, lo que ocurre por debajo es que se crean punteros a funciones dentro de cada instancia del objeto para saber a qué objeto saltar.
La implementación final, es una lista de punteros a funciones para cada función virtual que exista. Como se puede apreciar, tener esta lista de punteros tiene un coste, en el tamaño y en el tiempo de ejecución, ya que se necesita de un salto extra a la hora de llamar a la función. Esta lista de funciones se conoce comúnmente como VTABLE.

El atributo “final” se puede utilizar para indicar que una funcion o clase no van a poder ser sobrecargados:

In [11]:
class VirtualClass 
{
    public:
    VirtualClass(int initialScore) : score(initialScore)
    {
    }
    
    virtual void func() 
    {
        printf("padre");
    }
    
    private:
    int score;
};

//class VirtualSubClass2 final : public VirtualClass  //clase final
class VirtualSubClass2 : public VirtualClass 
{
    public:
    VirtualSubClass2() : VirtualClass(0)
    {
    } 
    
    virtual void func() override final //método final
    {
        printf("hijo\n");
    }
};

#### CONSTRUCTORES Y DESTRUCTORES

In [6]:
#include <cstdio>

class ClassWithDestructor {
public:
    ClassWithDestructor() {
        printf("Constructor\n");
    }

    ~ClassWithDestructor() {
        printf("Destructor\n");
    }
};

ClassWithDestructor* obj = new ClassWithDestructor();
delete obj;
  
return 0;

Constructor
Destructor


¿Qué ocurre si yo llamo al destructor de una clase padre? Tenemos un memory leak, el destructor del hijo
nunca se llama ya que no sabe cual es el tipo real de padre, y por lo tanto no lo llama. Para evitar esto, tenemos que marcar el destructor como virtual.

REGLA DE ORO\
En cuanto tengamos herencia, siempre debemos crear un destructor virtual en el padre. Aunque no tenga destructor constructor
custom, crearemos leaks sin darnos cuenta.

Hay ciertas reglas sobre qué ocurre cuando no ponemos constructores/destructores en nuestra clase/struct:
- Si no ponemos nada, C++ automáticamente creará un constructor por defecto, otro de copia, un destructor y operador de asignación.
- Si escribimos un constructor, el compilador no generará uno por defecto, pero si generará el resto.
- Existe un tercer tipo de constructor y asignación llamado move constructor que hace mucho más complicadas las reglas afectando al constructor de
copia y operator=, de momento haremos como si no existiera.

Podemos realizar explícitamente lo que hace C++ a la hora de crear los constructores por nosotros a mano usando “default”

In [1]:
struct Ejemplo 
{
    Ejemplo() = default;
    Ejemplo(int _a) : a(_a) {}
    int a;    
}

También podemos evitar que el compilador nos cree un constructor usando “delete”:

In [1]:
struct Ejemplo 
{
    Ejemplo() = delete;
    int a;
}

## CASTS

#### CASTS DE C

In [4]:
float a = 10.3f;
int b = (int)a;
printf("%i", b);

10

#### STATIC_CAST

Permite conversión entre primitivos con pérdida, conversiones explícitas y conversiones entre clases que tengan relación de herencia.
La conversión entre Padre->Hijo no realiza ninguna comprobación de tipo en runtime.

In [2]:
class Padre { public: Padre() { printf("Soy Padre\n"); }};

//Call both constructors
class Hijo : public Padre { public: Hijo() { printf("Soy hijo\n"); }};

float a = 10;
int b = static_cast<int>(a); //Conversión con pérdida
printf("%i\n", b);

const float c = a;
const float d = static_cast<float>(a); //Conversión implícita
printf("%f\n", d);

printf("\n--- New hijo\n");
Hijo* hijo = new Hijo(); 

printf("\n--- Padre = hijo\n");
Padre* padre = hijo;

printf("\n--- Padre2 = cast(hijo)\n");
Padre* padre2 = static_cast<Padre*>(hijo); //Conversión implícita

printf("\n--- Hijo2 = cast(padre)\n");
Hijo* hijo2 = static_cast<Hijo*>(padre); //Conversión entre clases relacionadas


10
10.000000

--- New hijo
Soy Padre
Soy hijo

--- Padre = hijo

--- Padre2 = cast(hijo)

--- Hijo2 = cast(padre)


#### REINTERPR_CAST

Este tipo de conversión menos restrictivo (y peligroso) que static_cast, permite convertir entre punteros de distinto tipo no relacionados. Al igual que static_cast, no permite conversions que quiten const

In [5]:
int a = 23;
int* intPtr = &a;
float* fPtr = reinterpret_cast<float*>(intPtr);
printf("%f \n", *fPtr);

void* ptr = &a;
int* otherA = reinterpret_cast<int*>(ptr);
printf("%i \n", *otherA);

0.000000 
23 


#### CONST_CAST

Esta conversión permite quitar const de un puntero o una referencia. Es una operación peligrosa, aunque pocos, tiene sus casos de uso. Nota: No se puede quitar el const si no es puntero o referencia. Ej: No se puede hacer const_cast<MyObj>()

In [15]:
int a = 10;
const int* ptrA = &a;
int* ptrANoConst = const_cast<int*>(ptrA);

//ptrA = 34;  //No podemos hacer esta asignación
*ptrANoConst = 34;
printf("%i", *ptrA);

34

#### DYNAMIC_CAST

Se utiliza para convertir objetos con relacion de herencia entre ellos, a diferencia de static_cast, si realiza una comprobacion de que los tipos sean validos. Si el tipo no es correcto, devolvera un nullptr. Tambien a diferencia de los static_cast, en este caso no tiene sentido usar & ya que no podemos comprobar si es nullptr.

Nota, para poder usar dynamic_cast, al igual que type_id(), es necesario habilitar RTTI (RunTime Type Information) en el compilador de C++.
El motivo para ello es que para poder usar dynamic cast, es necesario añadir informacion extra en todos las clases para saber cual es su verdadero tipo. Esto es análogo a las VTABLE, y en ocasiones puede ser optimizado por el compilador. Unreal Engine por ejemplo tiene desactivado RTTI y tiene su propio dynamic_cast, llamado Cast().

In [5]:
class Padre 
{ 
    public: 
        Padre() { printf("Soy Padre\n"); };
        virtual void virtualFunction() {};   //Necesitamos un método virtual 
};                                           //para que funcione el dynamic_cast
class Hijo : public Padre 
{ 
    public: 
        Hijo() { printf("Soy hijo\n"); };
};

//Padre padre;
//Hijo* hijo = dynamic_cast<Hijo*>(&padre); //Hijo es nullptr

Padre* padrePtr = new Hijo();
Hijo* hijoPtr = dynamic_cast<Hijo*>(padrePtr); // hijo == padre

Soy Padre
Soy hijo
